In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches
import json
from io import BytesIO
import os

In [2]:
import urllib.parse as up

uri = "hidden"
result = up.urlparse(uri)

DB_CONFIG = {
    "dbname": result.path.lstrip("/"),          # 'postgres'
    "user":   result.username,                  # 'postgres'
    "password": result.password,                # 'MY_SECRET_PASSWORD'
    "host":   result.hostname,                  # 'db.abcd1234.supabase.co'
    "port":   result.port or 5432
}

In [3]:
# List of participant names you want to generate reports for
PARTICIPANT_NAMES = [f'P{i}' for i in range(1, 11)]

In [6]:
def get_db_connection():
    """Establishes and returns a database connection."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except psycopg2.OperationalError as e:
        print(f"Error: Could not connect to the database. Please check DB_CONFIG.")
        print(f"Details: {e}")
        return None

def fetch_lookup_data(conn):
    """Fetches static data like KC names, Goal names, etc., into dictionaries for easy lookup."""
    print("Fetching lookup data (KCs, Goals, Metrics)...")
    kcs = pd.read_sql("SELECT id, kc_identifier, name FROM kcs", conn).set_index('id').to_dict('index')
    goals = pd.read_sql("SELECT id, name, description FROM goals", conn).set_index('id').to_dict('index')
    metrics = pd.read_sql("SELECT id, name FROM metrics", conn).set_index('id').to_dict('index')
    return {"kcs": kcs, "goals": goals, "metrics": metrics}

def create_metric_plot(df_metric_history, lookup_tables):
    """Creates a line chart of metric performance over decisions and returns it as an image stream."""
    if df_metric_history.empty:
        return None

    # <<< FIX: Ensure 'value' column is numeric
    df_metric_history['value'] = pd.to_numeric(df_metric_history['value'], errors='coerce')
    df_metric_history.dropna(subset=['value'], inplace=True)
    
    # Replace metric_id with metric_name for the legend
    df_metric_history['metric_name'] = df_metric_history['metric_id'].apply(lambda x: lookup_tables['metrics'].get(x, {}).get('name', f'Metric {x}'))
    
    fig, ax = plt.subplots(figsize=(8, 4))
    
    # Pivot data to plot each metric as a separate line
    pivot_df = df_metric_history.pivot_table(index='decision_number', columns='metric_name', values='value', aggfunc='last')
    pivot_df.plot(kind='line', marker='o', ax=ax)
    
    ax.set_title("Metric Performance Over Decisions")
    ax.set_xlabel("Decision Number")
    ax.set_ylabel("Metric Value")
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend(title='Metrics')
    plt.tight_layout()
    
    # Save plot to a memory buffer
    img_buffer = BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    plt.close(fig)
    
    return img_buffer

def parse_and_plot_kcs(dialogue_history_json, target_kcs, lookup_tables):
    """Parses dialogue history to track KC scores and generates a plot."""
    if not dialogue_history_json:
        return None, "No dialogue history available for this goal."

    # Initialize progress tracker
    kc_progress = {kc_id: [0] for kc_id in target_kcs}
    
    try:
        # Loop through the conversation turns
        for i, turn in enumerate(dialogue_history_json):
            if turn.get('role') == 'user' and 'User chose decision index' in turn.get('content', ''):
                decision_index_str = turn['content'].split(': ')[-1].split(':')[0].strip() # Handle cases like "User chose decision index: 0: '...'"
                decision_index = int(decision_index_str)
                
                # The assistant's turn with the question is the one before the user's choice
                assistant_turn = dialogue_history_json[i-1]
                assistant_content = json.loads(assistant_turn['content'])
                
                options = assistant_content.get('decisionPoint', {}).get('options', [])
                if decision_index < len(options):
                    chosen_option = options[decision_index]
                    kc_impacts = chosen_option.get('kc_impacts', [])
                    
                    # Update scores for all KCs before applying new impact
                    for kc_id in kc_progress:
                        last_score = kc_progress[kc_id][-1]
                        kc_progress[kc_id].append(last_score)

                    # Apply the new impacts
                    for impact in kc_impacts:
                        kc_identifier = impact['kc_identifier']
                        score_change = impact.get('score', 0)
                        
                        # Find the kc_id from the identifier
                        for k_id, k_info in lookup_tables['kcs'].items():
                            if k_info['kc_identifier'] == kc_identifier:
                                if k_id in kc_progress:
                                    kc_progress[k_id][-1] += score_change
                                break

    except (json.JSONDecodeError, KeyError, IndexError, TypeError, ValueError) as e:
        return None, f"Could not parse KC data from dialogue history. Error: {e}"

    if not any(len(v) > 1 for v in kc_progress.values()):
        return None, "No scorable KC decisions were made in this goal."

    # Prepare DataFrame for plotting
    plot_data = []
    for kc_id, scores in kc_progress.items():
        kc_name = lookup_tables['kcs'].get(kc_id, {}).get('name', f'KC {kc_id}')
        for i, score in enumerate(scores):
            plot_data.append({'decision': i, 'kc_name': kc_name, 'score': score})
    
    df_plot = pd.DataFrame(plot_data)

    fig, ax = plt.subplots(figsize=(8, 4))
    pivot_df = df_plot.pivot_table(index='decision', columns='kc_name', values='score')
    pivot_df.plot(kind='line', marker='o', ax=ax)
    
    ax.set_title("Knowledge Component (KC) Learning Curve")
    ax.set_xlabel("Decision Number")
    ax.set_ylabel("Cumulative KC Score")
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend(title='Knowledge Components')
    plt.tight_layout()
    
    img_buffer = BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    plt.close(fig)
    
    return img_buffer, "KC learning curve generated."


def add_decision_path_to_doc(doc, dialogue_history_json, lookup_tables):
    """Parses dialogue history and adds the decision path to the Word document."""
    if not dialogue_history_json:
        doc.add_paragraph("No dialogue history available.")
        return
        
    doc.add_heading("C. Decision Path & Rationale", level=3)
    
    decision_counter = 1
    try:
        for i, turn in enumerate(dialogue_history_json):
            if turn.get('role') == 'assistant':
                assistant_content = json.loads(turn['content'])
                decision_point = assistant_content.get('decisionPoint')
                if decision_point and 'question' in decision_point and i + 1 < len(dialogue_history_json):
                    # This is a question turn. The user's answer is next.
                    user_turn = dialogue_history_json[i+1]
                    if 'User chose decision index' in user_turn.get('content', ''):
                        decision_index_str = user_turn['content'].split(': ')[-1].split(':')[0].strip() # Handle cases like "User chose decision index: 0: '...'"
                        decision_index = int(decision_index_str)
                        chosen_option = decision_point['options'][decision_index]
                        
                        doc.add_paragraph(f"Decision {decision_counter}:", style='List Bullet')
                        p = doc.add_paragraph()
                        p.add_run("Scenario: ").bold = True
                        p.add_run(decision_point['question'])

                        p = doc.add_paragraph()
                        p.add_run("Participant's Choice: ").bold = True
                        p.add_run(chosen_option['text'])
                        
                        kc_impact_str = ", ".join([f"{imp['score']} to {imp['kc_identifier']}" for imp in chosen_option.get('kc_impacts', [])])
                        p = doc.add_paragraph()
                        p.add_run("Immediate KC Impact: ").bold = True
                        p.add_run(kc_impact_str if kc_impact_str else "None")
                        
                        doc.add_paragraph() # Add some space
                        decision_counter += 1

    except (json.JSONDecodeError, KeyError, IndexError, TypeError, ValueError) as e:
        doc.add_paragraph(f"Error parsing decision path: {e}")

def generate_report_for_user(user_name, conn, lookup_tables):
    """Main function to generate a single report for a given user."""
    print(f"\n--- Generating report for {user_name} ---")
    
    # 1. Fetch all data for this specific user
    user_info = pd.read_sql("SELECT id FROM users WHERE name = %(name)s", conn, params={"name": user_name})
    if user_info.empty:
        print(f"User '{user_name}' not found in the database. Skipping.")
        return
    user_id = user_info['id'].iloc[0]
    
    df_user_goals = pd.read_sql("SELECT * FROM user_goals WHERE user_id = %(user_id)s ORDER BY goal_id", conn, params={"user_id": str(user_id)})
    df_metric_history = pd.read_sql("SELECT * FROM user_metric_history WHERE user_id = %(user_id)s", conn, params={"user_id": str(user_id)})
    df_kc_scores = pd.read_sql("SELECT * FROM user_kc_scores WHERE user_id = %(user_id)s", conn, params={"user_id": str(user_id)})
    df_metric_scores = pd.read_sql("SELECT * FROM user_metric_scores WHERE user_id = %(user_id)s", conn, params={"user_id": str(user_id)})
    df_goal_kcs = pd.read_sql("SELECT * FROM goal_kcs", conn)
    
    # <<< FIX: Convert score columns to numeric types right after fetching data
    df_kc_scores['current_score'] = pd.to_numeric(df_kc_scores['current_score'], errors='coerce')
    df_kc_scores.dropna(subset=['current_score'], inplace=True)
    
    df_metric_scores['current_value'] = pd.to_numeric(df_metric_scores['current_value'], errors='coerce')
    df_metric_scores.dropna(subset=['current_value'], inplace=True)

    # 2. Start creating the Word Document
    doc = Document()
    doc.add_heading('AI Entrepreneurship Game: Learning Report', 0)
    doc.add_heading(f"Participant: {user_name}", level=1)
    doc.add_paragraph(f"Participant ID: {user_id}")
    
    # 3. Section 1: Executive Summary
    doc.add_heading("1. Executive Summary & Overall Performance", level=2)
    
    # Final Metrics
    doc.add_paragraph().add_run("Final Key Metrics:").bold = True
    if df_metric_scores.empty:
        doc.add_paragraph("No final metric scores available.", style='List Bullet')
    else:
        for _, row in df_metric_scores.iterrows():
            metric_name = lookup_tables['metrics'].get(row['metric_id'], {}).get('name', f'Metric {row["metric_id"]}')
            doc.add_paragraph(f"{metric_name}: {row['current_value']:.2f}", style='List Bullet')
        
    # KC Strengths/Weaknesses
    df_kc_scores_named = df_kc_scores.copy()
    df_kc_scores_named['kc_name'] = df_kc_scores_named['kc_id'].apply(lambda x: lookup_tables['kcs'].get(x, {}).get('name', f'KC {x}'))
    
    doc.add_paragraph().add_run("Key Strengths (Top KCs):").bold = True
    if df_kc_scores_named.empty:
        doc.add_paragraph("No KC scores available.", style='List Bullet')
    else:
        for _, row in df_kc_scores_named.nlargest(3, 'current_score').iterrows():
            doc.add_paragraph(f"{row['kc_name']} - Score: {row['current_score']}", style='List Bullet')

    doc.add_paragraph().add_run("Areas for Improvement (Lowest KCs):").bold = True
    if df_kc_scores_named.empty:
        doc.add_paragraph("No KC scores available.", style='List Bullet')
    else:
        for _, row in df_kc_scores_named.nsmallest(3, 'current_score').iterrows():
            doc.add_paragraph(f"{row['kc_name']} - Score: {row['current_score']}", style='List Bullet')
        
    # 4. Section 2: Goal-by-Goal Analysis
    doc.add_heading("2. Detailed Goal-by-Goal Analysis", level=2)

    if df_user_goals.empty:
        doc.add_paragraph("This participant has not attempted any goals yet.")
    else:
        for _, goal_row in df_user_goals.iterrows():
            goal_id = goal_row['goal_id']
            goal_name = lookup_tables['goals'].get(goal_id, {}).get('name', f'Goal {goal_id}')
            
            doc.add_heading(f"Goal: {goal_name}", level=3)
            doc.add_paragraph(f"Status: {goal_row['status']}, Attempts: {goal_row['attempts_for_current_goal_cycle']}")
            
            # A. Metric Performance Curve
            doc.add_heading("A. Metric Performance Curve", level=4)
            df_goal_metrics = df_metric_history[df_metric_history['goal_id'] == goal_id]
            metric_plot_img = create_metric_plot(df_goal_metrics, lookup_tables)
            if metric_plot_img:
                doc.add_picture(metric_plot_img, width=Inches(6.0))
            else:
                doc.add_paragraph("No metric history recorded for this goal.")

            # B. KC Learning Curve
            doc.add_heading("B. Knowledge Component (KC) Learning Curve", level=4)
            target_kcs_for_goal = df_goal_kcs[df_goal_kcs['goal_id'] == goal_id]['kc_id'].tolist()
            kc_plot_img, message = parse_and_plot_kcs(goal_row['dialogue_history'], target_kcs_for_goal, lookup_tables)
            if kc_plot_img:
                doc.add_picture(kc_plot_img, width=Inches(6.0))
            else:
                doc.add_paragraph(message)
                
            # C. Decision Path
            add_decision_path_to_doc(doc, goal_row['dialogue_history'], lookup_tables)

    # 5. Section 3: Final Entrepreneurial Profile (Simplified logic)
    doc.add_heading("3. Final Entrepreneurial Profile", level=2)
    
    playstyle = "Balanced Entrepreneur"
    if not df_kc_scores_named.empty:
        top_kc = df_kc_scores_named.nlargest(1, 'current_score')
        if not top_kc.empty:
            top_kc_name = top_kc['kc_name'].iloc[0]
            if any(style in top_kc_name for style in ['Trust', 'Reputation', 'Ethical']):
                playstyle = "The Community Builder"
            elif any(style in top_kc_name for style in ['Revenue', 'Pricing', 'profit']):
                playstyle = "The Profit Maximizer"
            
    doc.add_paragraph().add_run("Identified Playstyle: ").bold = True
    doc.add_paragraph(playstyle)
    doc.add_paragraph().add_run("Recommendations:").bold = True
    doc.add_paragraph("Based on the KC scores, this participant could benefit from focusing on scenarios that challenge their lower-scoring skills to develop a more well-rounded entrepreneurial profile.")

    # 6. Save the document
    file_name = f"Learning_Report_{user_name}.docx"
    doc.save(file_name)
    print(f"Successfully generated report: {file_name}")

In [7]:
if __name__ == "__main__":
    conn = get_db_connection()
    if conn:
        try:
            lookup_data = fetch_lookup_data(conn)
            for name in PARTICIPANT_NAMES:
                generate_report_for_user(name, conn, lookup_data)
        finally:
            conn.close()
            print("\nDatabase connection closed.")

Fetching lookup data (KCs, Goals, Metrics)...

--- Generating report for P1 ---


C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  kcs = pd.read_sql("SELECT id, kc_identifier, name FROM kcs", conn).set_index('id').to_dict('index')
C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  goals = pd.read_sql("SELECT id, name, description FROM goals", conn).set_index('id').to_dict('index')
C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using

Successfully generated report: Learning_Report_P1.docx

--- Generating report for P2 ---


C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:174: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_info = pd.read_sql("SELECT id FROM users WHERE name = %(name)s", conn, params={"name": user_name})
C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:180: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_user_goals = pd.read_sql("SELECT * FROM user_goals WHERE user_id = %(user_id)s ORDER BY goal_id", conn, params={"user_id": str(user_id)})
C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:181: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2

Successfully generated report: Learning_Report_P2.docx

--- Generating report for P3 ---


...

Successfully generated report: Learning_Report_P8.docx

--- Generating report for P9 ---


C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:174: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_info = pd.read_sql("SELECT id FROM users WHERE name = %(name)s", conn, params={"name": user_name})
C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:180: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_user_goals = pd.read_sql("SELECT * FROM user_goals WHERE user_id = %(user_id)s ORDER BY goal_id", conn, params={"user_id": str(user_id)})
C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:181: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2

Successfully generated report: Learning_Report_P9.docx

--- Generating report for P10 ---


C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:174: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_info = pd.read_sql("SELECT id FROM users WHERE name = %(name)s", conn, params={"name": user_name})
C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:180: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_user_goals = pd.read_sql("SELECT * FROM user_goals WHERE user_id = %(user_id)s ORDER BY goal_id", conn, params={"user_id": str(user_id)})
C:\Users\eduar\AppData\Local\Temp\ipykernel_10632\1756634312.py:181: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2

Successfully generated report: Learning_Report_P10.docx

Database connection closed.
